# This notebook cleans the 2012/2016 county-level vote data

In [1]:
from IPython.display import display

import pandas as pd

### First clean up the 2012 data: Get counts for 4 main parties (Dem, Rep, Lib, Grn) + Other

In [2]:
# We begin with the raw Guardian 2012 data as scraped from that site
data_2012 = pd.read_csv('2012_data_Guardian.csv')
display(data_2012.head())

# Convert 2 columns "Party[.x] <party>, Votes <count>" into one column "<party> <count>" for the 4 major parties
parties = ['Dem', 'GOP', 'Lib', 'Grn']
for party in parties:
    data_2012["{}".format(party)] = 0
    for tag in ('', '.1', '.2', '.3', '.4'):
        mask = data_2012["Party{}".format(tag)] == party
        data_2012.loc[mask, "{}".format(party)] = data_2012[mask]["Votes{}".format(tag)]
data_2012 = data_2012.drop(data_2012.columns[[0] + list(range(5, 15))], axis=1)

# For 2012 a 0 FIPS code is a state aggregation, which we will ignore for now
data_2012 = data_2012.drop(data_2012[data_2012["FIPS Code"] == 0].index)
# Ignore PR as an outlier
data_2012 = data_2012.drop(data_2012[data_2012["State Postal"] == 'PR'].index)
# Lower case on the county string to help repair broken entries
data_2012['County Name'] = data_2012['County Name'].str.lower()

# Add a year column and redo columns a bit (reorder, rename)
data_2012["Year"] = 2012
# Convert vote counts to int
data_2012[parties] = data_2012[parties].astype(int)
# Simplify column names
data_2012 = data_2012.rename(columns={"FIPS Code": "FIPS", "TOTAL VOTES CAST": "Total Votes", "County Name": "County",
                              "State Postal": "State", "GOP": "Rep"})
data_2012 = data_2012[["Year", "FIPS", "State", "County", "Total Votes", "Dem", "Rep", "Lib", "Grn"]]
data_2012.head()

,Unnamed: 0,State Postal,FIPS Code,County Name,TOTAL VOTES CAST,Party,Votes,Party.1,Votes.1,Party.2,Votes.2,Party.3,Votes.3,Party.4,Votes.4
0,0,AK,0,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN
1,1,AK,2000,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN
2,2,AL,0,Alabama,2064699,Dem,793620,GOP,1252453,Ind,2961.0,Ind,12283.0,Ind,3382.0
3,3,AL,1001,Autauga,23909,Dem,6354,GOP,17366,Ind,31.0,Ind,136.0,Ind,22.0
4,4,AL,1003,Baldwin,84988,Dem,18329,GOP,65772,Ind,122.0,Ind,597.0,Ind,168.0


,Year,FIPS,State,County,Total Votes,Dem,Rep,Lib,Grn
1,2012,2000,AK,alaska,220596,91696,121234,5539,2127
3,2012,1001,AL,autauga,23909,6354,17366,0,0
4,2012,1003,AL,baldwin,84988,18329,65772,0,0
5,2012,1005,AL,barbour,11459,5873,5539,0,0
6,2012,1007,AL,bibb,8391,2200,6131,0,0


### Now for 2016

In [8]:
# Same deal, begin with raw scraped data for 2016 from Townhall
data_2016 = pd.read_csv('2016_data_Townhall.csv')
display(data_2016.head())

# Do some quick renaming so the 2016 and 2012 data match column names
data_2016 = data_2016.rename(columns={"state_abbr": "State", "county_name": "County", "votes_total_2016": "Total Votes"})
data_2016 = data_2016.drop(['Unnamed: 0'], axis=1)
data_2016 = data_2016.drop(data_2016[data_2016["State"] == 'PR'].index)
data_2016['County'] = data_2016['County'].str.lower()

# Fix some stupid mistakes in the data
#data_2016 = data_2016.set_value(2802, 3, "bedford")

# Create the desired output structure (one line per country) with totals
merging = data_2016.groupby(['State', 'County'], sort=False).sum().reset_index()

# Now get the party vote counts
parties = ['DEM', 'GOP', 'LIB', 'GRN']
for party in parties:
    to_merge = data_2016[data_2016['party'] == party].set_index(['State', 'County'])
    to_merge = to_merge.rename(columns={"Total Votes": party}).drop('party', axis=1)
    merging = merging.join(to_merge, on=['State', 'County'], how='left')
    
merging[parties + ["Total Votes"]] = merging[parties + ["Total Votes"]].fillna(value=0).astype(int)

# Get starting FIPS from 2012 data 
to_merge = data_2012.set_index(['State', 'County'])[['FIPS']]
merging = merging.join(to_merge, on=['State', 'County'], how='left')

# Finished with merges, rename back to 2016
data_2016 = merging

# Add year and fix columns
data_2016.insert(0, "Year", 2016)
data_2016.insert(1, "Fips", data_2016['FIPS'])
data_2016 = data_2016.drop('FIPS', axis=1)
data_2016 = data_2016.rename(columns={"Fips": "FIPS", "DEM": "Dem", "GOP": "Rep", "LIB": "Lib", "GRN": "Grn"})
data_2016[["FIPS"]] = data_2016[["FIPS"]].fillna(value=0).astype(int)
data_2016.head()

,Unnamed: 0,state_abbr,county_name,party,votes_total_2016
0,1,AL,Autauga,GOP,18110.0
1,2,AL,Autauga,DEM,5908.0
2,3,AL,Autauga,IND,538.0
3,4,AL,Autauga,IND,105.0
4,5,AL,Baldwin,GOP,72780.0


,Year,FIPS,State,County,Total Votes,Dem,Rep,Lib,Grn
0,2016,1001,AL,autauga,24661,5908,18110,0,0
1,2016,1003,AL,baldwin,94090,18409,72780,0,0
2,2016,1005,AL,barbour,10390,4848,5431,0,0
3,2016,1007,AL,bibb,8748,1874,6733,0,0
4,2016,1009,AL,blount,25384,2150,22808,0,0


#### Now do some repairs on the data
* Using a FIPS database, fix some of the missing codes and incorrect county names

In [11]:
# Load the FIPS data

# Apparently Alaska has a bunch of counties but just aggregates by state for votes?
county_exclusions = [('kalawao', 'HI')]
# Exclude non-state regions that are included in FIPS
non_state = ['PR', 'AS', 'MP', 'GU', 'VI', 'UM']
# Alaska also has a bunch of buroughs, but votes are counted at state level
state_exclusions = non_state + ['AK']
# 5 'city-type' counties for Virginia that collide unless treated separately
va_city_exceptions = ['bedford city', 'fairfax city', 'franklin city', 'richmond city', 'roanoke city']

fips_codes = {}
with open('raw_census_data/FIPS') as fh:
    fips_data = fh.readlines()
    for line in fips_data:
        entry = line.split(',')
        # Remove the extra "County"/"Parish" from the names
        county_name = entry[3].replace(' County', '').replace(' Parish', '').lower()
        if entry[0] in state_exclusions or (county_name, entry[0]) in county_exclusions:
            continue
        elif entry[0] == 'VA' and county_name not in va_city_exceptions:
            # Virginia also has cities as counties
            county_name = county_name.replace(' city', '')
        key = (county_name, entry[0])
        if key in fips_codes:
            print ("Duplicate {}".format(key))
        fips_codes[key] = int(entry[1] + entry[2])
        
# Shannon county, SD has changed in 2015, reflected in the 2016 vote data
fips_codes.pop(("shannon", "SD"))
fips_codes[("oglala lakota", "SD")] = 46102
# Bedford City (county), VA is no longer a county
fips_codes.pop(("bedford city", "VA"))
# Add in Alaska
fips_codes[("alaska", "AK")] = 2000
print("{} FIPS codes loaded".format(len(fips_codes)))

3113 FIPS codes loaded


In [12]:
# Function to apply the FIPS lookup to find incorrect or missing values
def analyze_repair_FIPS(df, fips_codes, mistakes=(), output=False):
    manual, empty, renamed, recoded = 0, 0, 0, 0
    # First change passed-in fixes
    for identifiers, repairs in mistakes:
        mask = 1
        for key, value in identifiers.items():
            mask &= df[key] == value
        try:
            index = df[mask].index.values[0]
            for key, value in repairs.items():
                df.set_value(index, key, value)
            manual += 1
            if output:
                print ("Identifiers {} yield index {}: using repairs {}".format(identifiers, index, repairs))
        except:
            pass
    # Now correct entries using the loaded FIPS codes
    for county, code in list(fips_codes.items())[:]:
        curr = df[(df["State"] == county[1]) & (df["County"] == county[0])]
        if curr.empty:
            try:
                check = df[df["FIPS"] == code]['County']
                if output:
                    print("Changing {} to {} for FIPS {} in State {}".format(check.iloc[0], county[0], code, county[1]))
                index = check.index.values.tolist()[0]
                df.set_value(index, 'County', county[0])
                renamed += 1
            except:
                empty += 1
                print("Empty: {} {}".format(county, code))
        elif curr.iloc[0]['FIPS'] != code:
            if output:
                print("Changing {} FIPS {}->{}".format(county, curr.iloc[0]['FIPS'], code))
            recoded += 1
            index = curr.index.values.tolist()[0]
            df.set_value(index, 'FIPS', code)
            
    print("{} manual fixes | {} county names fixed | {} FIPS codes fixed | {} still empty".format(manual, renamed, recoded, empty))

In [13]:
# 2012
mistakes_2012 = [({"County": "shannon", "State": "SD"}, {"County": "oglala lakota", "FIPS": 46102}),
                 ({"State": "VA", "County": "bedford city"}, {"FIPS": 51019})]
analyze_repair_FIPS(data_2012, fips_codes, mistakes_2012)

2 manual fixes | 41 county names fixed | 12 FIPS codes fixed | 0 still empty


In [14]:
#2016
mistakes_2016 = [({"State": "MO", "County": "sainte genevieve"}, {"County": "ste. genevieve", "FIPS": 29186}),
                 ({"State": "MO", "County": "st. louis"}, {"County": "st. louis city", "FIPS": 29510}),
                 ({"State": "MO", "County": "st. louis co."}, {"County": "st. louis", "FIPS": 29189}),
                 ({"State": "DC", "County": "washington"}, {"County": "district of columbia", "FIPS": 11001}),
                 ({"State": "MD", "County": "baltimore county"}, {"County": "baltimore", "FIPS": 24005}), 
                 ({"State": "IL", "County": "la salle"}, {"County": "lasalle", "FIPS": 17099}),
                 ({"State": "VA", "County": "charles city"}, {"County": "charles"}),
                 ({"State": "VA", "County": "james city"}, {"County": "bedford"}),
                 ({"State": "VA", "County": "bedford co."}, {"County": "bedford"}),
                 ({"State": "VA", "County": "fairfax co."}, {"County": "fairfax city"}),
                 ({"State": "VA", "County": "franklin co."}, {"County": "franklin city"}),
                 ({"State": "VA", "County": "richmond co."}, {"County": "richmond city"}),
                 ({"State": "VA", "County": "roanoke co."}, {"County": "roanoke city"}),
                ]
analyze_repair_FIPS(data_2016, fips_codes, mistakes_2016)

13 manual fixes | 1 county names fixed | 56 FIPS codes fixed | 0 still empty


In [15]:
# Finally, the 2012 data has several New England states which have broken down their counties into tiny voting areas
# Since the census data we work with later is only county-level, and the 2016 voting data isn't on this small scale, we should fix it
data_2012 = data_2012.groupby(['FIPS'], sort=False).sum().reset_index()

fips_linkup = pd.DataFrame([{"FIPS": code, "State": key[1], "County": key[0]} for key, code in fips_codes.items()]).set_index('FIPS')
data_2012 = data_2012.join(fips_linkup, on=['FIPS'], how='left')
data_2012 = data_2012[["Year", "FIPS", "State", "County", "Total Votes", "Dem", "Rep", "Lib", "Grn"]]

In [17]:
# Final output of clean, county-level voting data
print ("Total county entries | 2012: {}  2016: {}  FIPS codes: {}".format(len(data_2012), len(data_2016), len(fips_codes)))
data_2016.to_csv('2012_clean_votes.csv', sep=',')
data_2016.to_csv('2016_clean_votes.csv', sep=',')

Total county entries | 2012: 3113  2016: 3113  FIPS codes: 3113


In [ ]:
# In order to detect the issue with New England sub-county voting, I ran the following

def analyse_voting_blocks(df):
    ct = 0
    considered, fips_set, state_set = set(), set(), set()
    for fc in df['FIPS'].values:
        if fc in fips_set and fc not in considered:
            matching_entries = df[df['FIPS'] == fc]
            state = matching_entries['State'].iloc[0]
            if not all(matching_entries['State'].values == state):
                print ("Not all states match")
                ct += 1
                if ct <= 2:
                    display(matching_entries)
                considered |= set(matching_entries['FIPS'].values)
            else:
                if state not in state_set:
                    print("{} has broken down county voting blocks".format(state))
                    state_set.add(state)
                considered |= set(matching_entries['FIPS'].values)
                continue
        fips_set.add(fc)

In [ ]:
analyse_voting_blocks(data_2012)
analyse_voting_blocks(data_2016)